# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

&emsp;&emsp;<font face="仿宋">在案例的第二部分中，我们详细介绍了常用特征转化方法，其中有些是模型训练之必须，如自然数编码、独热编码，而有些方法则是以提高数据质量为核心、在大多数时候都是作为模型优化的备选方法，如连续变量分箱、数据标准化等。当然，在此之后，我们首先尝试构建一些可解释性较强的模型来进行用户流失预测，即采用逻辑回归和决策树模型来进行预测，并同时详细介绍了两种模型在实战中的调优技巧，在最终模型训练完成后，我们也重点讨论了关于两种可解释性模型建模结果的解释方法。

&emsp;&emsp;<font face="仿宋">从理论上来说，树模型的判别能力是要强于逻辑回归的，但在上一节最后的建模结果中我们发现两个模型的建模并无显著差别，预测准确率都维持在79%-80%之间，这或许说明很多逻辑回归无法正确判别的样本决策树模型也无法判别，据此我们推测，这是一个“入门容易、精通较难”的数据集。当然，如果我们进一步尝试其他“更强”的集成学习算法，如随机森林、XGB、CatBoost等，在当前数据集上的建模结果和逻辑回归也并无太大差异，因此我们亟需通过特征工程方法进一步提升数据集质量，进而提升最终模型效果。

&emsp;&emsp;<font face="仿宋">当然，哪怕是复杂模型在当前数据集上表现出了更好的效果，采用特征工程方法提升数据质量仍是优化建模结果必不可少的部分，正如时下流行的描述那样，“数据质量决定模型上界，而建模过程只是不断逼近这个上界”，特征工程中的一系列提高数据质量的方法、无论是在工业界实践中还是各大顶级竞赛里，都已然成了最为重要的提升模型效果的手段。

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gwllgk4wgqj31hr0u0wh4.jpg" alt="image-20211112170651500" style="zoom:15%;" />

&emsp;&emsp;<font face="仿宋">不过，所谓的通过特征工程方法提高数据质量，看似简单但实际操作起来却并不容易。其难点并不在于其中具体操作方法的理解，至少相比机器学习算法原理，特征工程的很多方法并不复杂，特征工程的最大难点在于配合模型与数据进行方法选择、以及各种方法的工程化部署实现。一方面，特征工程方法众多，需要根据实际情况“因地制宜”，但数据的情况千变万化，很多时候需要同时结合数据探索结论、建模人员自身经验以及对各种备选方法的熟悉程度，才能快速制定行之有效的特征工程策略；另一方面，很多特征工程方法不像机器学习算法有现成的库可以直接调用，很多方法、尤其是一些围绕当前数据集的定制方法，需要自己手动实现，而这个过程就对建模人员本身的代码编写能力及工程部署能力提出了更高的要求。总而言之，特征工程是一个实践高度相关的技术，这也是为何课程会在介绍案例的过程中同步介绍特征工程常用方法的原因。

&emsp;&emsp;<font face="仿宋">当然，从宽泛的角度来看，所有围绕数据集的数据调整工作都可以看成是特征工程的一部分，包括此前介绍的缺失值填补、数据编码、特征变换等，这些方法其实都能一定程度提升数据质量，而本节开始，我们将花费一整节的时间来讨论另一类特征工程方法：特征衍生与特征筛选。而该方法通过创建更多特征来提供更多捕捉数据规律的维度，从而提升模型效果。当然特征衍生也是目前公认的最为有效的、能够显著提升数据集质量方法。

# <center>Part 3.分组统计特征衍生策略

&emsp;&emsp;本阶开始我们将重点讨论特征工程中的特征衍生与特征筛选方法，并借此进一步提升模型效果。首先需要将此前的操作中涉及到的第三方库进行统一的导入：

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *

# re模块相关
import inspect, re

其中telcoFunc是自定义的模块，其内保存了此前自定义的函数和类，后续新增的函数和类也将逐步写入其中，telcoFunc.py文件随课件提供，需要将其放置于当前ipy文件同一文件夹内才能正常导入。

&emsp;&emsp;接下来导入数据并执行Part 1中的数据清洗步骤。

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

接下来即可直接带入数据进行特征衍生。

### 3.分组统计特征衍生

- 方法介绍

&emsp;&emsp;接下来，我们继续讨论另一种同样非常常用的特征衍生方法：分组统计特征衍生方法。所谓分组统计，顾名思义，就是A特征根据B特征的不同取值进行分组统计，统计量可以是均值、方差等针对连续变量的统计指标，也可以是众数、分位数等针对离散变量的统计指标，例如我们可以计算不同入网时间用户的平均月消费金额、消费金额最大值、消费金额最小值等，基本过程如下：

<center><img src="https://s2.loli.net/2022/01/20/6YzMGo9uI1Dm8rc.png" alt="image-20220120170224947" style="zoom:33%;" />

同样，该过程也并不复杂，在实际执行分组统计特征衍生的过程中（假设是A特征根据B特征的不同取值进行分组统计），有以下几点需要注意：

- 首先，一般来说A特征可以是离散变量也可以是连续变量，而B特征必须是离散变量，且最好是一些取值较多的离散变量（或者固定取值的连续变量），例如本数据集中的tenure字段，总共有73个取值。主要原因是如果B特征取值较少，则在衍生的特征矩阵中会出现大量的重复的行；      
- 其次，在实际计算A的分组统计量时，可以不局限于连续特征只用连续变量的统计量、离散特征只用离散的统计量，完全可以交叉使用，例如A是离散变量，我们也可以分组统计其均值、方差、偏度、峰度等，连续变量也可以统计众数、分位数等。很多时候，更多的信息组合有可能会带来更多的可能性；        
- 其三，有的时候分组统计还可以用于多表连接的场景，例如假设现在给出的数据集不是每个用户的汇总统计结果，而是每个用户在过去的一段时间内的行为记录，则我们可以根据用户ID对其进行分组统计汇总：        
<center><img src="https://s2.loli.net/2022/01/20/mE9hBtyaHpOIlvR.png" alt="image-20220120172058991" style="zoom:33%;" />      
- 其四，很多时候我们还会考虑进一步围绕特征A和分组统计结果进行再一次的四则运算特征衍生，例如用月度消费金额减去分组均值，则可以比较每一位用户与相同时间入网用户的消费平均水平的差异，围绕衍生特征再次进行衍生，我们将其称为统计演变特征，也是分组汇总衍生特征的重要应用场景：     
<center><img src="https://s2.loli.net/2022/01/20/Z1bgnjBui7VT4dI.png" alt="image-20220120172703371" style="zoom:33%;" />        

- 手动实现

&emsp;&emsp;接下来我们考虑分组汇总特征如何实现。这里我们可以优先考虑借助Pandas中的groupby方法来实现，首先简单回归groupby方法的基本使用，这里我们提取'tenure'、'SeniorCitizen'、'MonthlyCharges'三列来尝试进行单列聚合和多列聚合：

In [4]:
# 提取目标字段
colNames = ['tenure', 'SeniorCitizen', 'MonthlyCharges']

In [5]:
# 单独提取目标字段的数据集
features_temp = features[colNames]
features_temp.head(5)

,tenure,SeniorCitizen,MonthlyCharges
0,1,0,29.85
1,34,0,56.95
2,2,0,53.85
3,45,0,42.30
4,2,0,70.70


In [6]:
# 在不同tenure取值下计算其他变量分组均值的结果
features_temp.groupby('tenure').mean()

,SeniorCitizen,MonthlyCharges
tenure,,
0,0.000000,41.418182
1,0.140294,50.485808
2,0.180672,57.206303
3,0.125000,58.015000
4,0.147727,57.432670
...,...,...
68,0.130000,73.321000
69,0.136842,70.823158
70,0.142857,76.378992


In [7]:
# 在不同tenure取值下计算其他变量分组标准差的结果
features_temp.groupby('tenure').std()

,SeniorCitizen,MonthlyCharges
tenure,,
0,0.000000,23.831484
1,0.347575,24.714198
2,0.385557,25.180714
3,0.331549,26.783798
4,0.355842,26.362647
...,...,...
68,0.337998,30.267744
69,0.345504,33.730068
70,0.351407,30.993483


此外，我们还可以尝试多列聚合

In [11]:
# 在'tenure'、'SeniorCitizen'交叉取值分组下，计算组内月度消费金额均值
features_temp.groupby(['tenure', 'SeniorCitizen']).mean()

MonthlyCharges
tenure SeniorCitizen                
0      0                   41.418182
1      0                   48.329127
       1                   63.701744
2      0                   54.951538
       1                   67.431395
...                              ...
70     1                   87.017647
71     0                   72.497482
       1                   79.287097
72     0                   78.687745
       1                   91.668750

[145 rows x 1 columns]

当然，groupby也支持同时输入多个统计量进行汇总计算，此时推荐使用agg方法来进行相关操作:

In [8]:
colNames

['tenure', 'SeniorCitizen', 'MonthlyCharges']

In [9]:
# 分组汇总字段
colNames_sub = ['SeniorCitizen', 'MonthlyCharges']

In [10]:
# 创建空字典
aggs = {}

# 字段汇总统计量设置
for col in colNames_sub:
    aggs[col] = ['mean', 'min', 'max']

In [11]:
# 每个字段汇总统计信息
aggs

{'SeniorCitizen': ['mean', 'min', 'max'],
 'MonthlyCharges': ['mean', 'min', 'max']}

In [12]:
# 创建新的列名称
cols = ['tenure']

for key in aggs.keys():
    cols.extend([key+'_'+'tenure'+'_'+stat for stat in aggs[key]])

In [13]:
cols

['tenure',
 'SeniorCitizen_tenure_mean',
 'SeniorCitizen_tenure_min',
 'SeniorCitizen_tenure_max',
 'MonthlyCharges_tenure_mean',
 'MonthlyCharges_tenure_min',
 'MonthlyCharges_tenure_max']

而这里的列表表达式，返回结果如下：

In [14]:
[key+'_'+'tenure'+'_'+stat for stat in aggs[key]]

['MonthlyCharges_tenure_mean',
 'MonthlyCharges_tenure_min',
 'MonthlyCharges_tenure_max']

这也是为何使用extend方法的原因。接下来我们创建新特征：

In [15]:
features_new = features_temp.groupby('tenure').agg(aggs).reset_index()
features_new.head(5)

tenure SeniorCitizen         MonthlyCharges               
                  mean min max           mean    min     max
0      0      0.000000   0   0      41.418182  19.70   80.85
1      1      0.140294   0   1      50.485808  18.80  102.45
2      2      0.180672   0   1      57.206303  18.75  104.40
3      3      0.125000   0   1      58.015000  18.80  107.95
4      4      0.147727   0   1      57.432670  18.85  105.65

In [16]:
# 重新设置列名称
features_new.columns = cols
features_new.head(5)

,tenure,SeniorCitizen_tenure_mean,SeniorCitizen_tenure_min,SeniorCitizen_tenure_max,MonthlyCharges_tenure_mean,MonthlyCharges_tenure_min,MonthlyCharges_tenure_max
0,0,0.000000,0,0,41.418182,19.70,80.85
1,1,0.140294,0,1,50.485808,18.80,102.45
2,2,0.180672,0,1,57.206303,18.75,104.40
3,3,0.125000,0,1,58.015000,18.80,107.95
4,4,0.147727,0,1,57.432670,18.85,105.65


当然，在创建完统计汇总信息后，还需要以tenure为主键和原始数据集进行拼接，此时需要使用merge函数进行操作：

In [17]:
df_temp = pd.merge(features, features_new, how='left',on='tenure')
df_temp.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,SeniorCitizen_tenure_mean,SeniorCitizen_tenure_min,SeniorCitizen_tenure_max,MonthlyCharges_tenure_mean,MonthlyCharges_tenure_min,MonthlyCharges_tenure_max
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,Yes,Electronic check,29.85,29.85,0.140294,0,1,50.485808,18.80,102.45
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,Mailed check,56.95,1889.50,0.184615,0,1,69.644615,19.60,116.25
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,Yes,Mailed check,53.85,108.15,0.180672,0,1,57.206303,18.75,104.40
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,Bank transfer (automatic),42.30,1840.75,0.196721,0,1,71.245902,18.85,115.65
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,Yes,Electronic check,70.70,151.65,0.180672,0,1,57.206303,18.75,104.40


- 常用统计量补充

&emsp;&emsp;这里我们对所有分组统计过程中可能用到的统计量进行汇总，需要注意的是，在进行分组汇总统计时，我们往往会无差别的进行尽可能多的统计量进行计算，只在针对离散或连续变量时进行统计量设置的些许调整。可用于连续性变量的统计量如下：

    - mean/var：均值、方差；
    - max/min：最大值、最小值；
    - skew：数据分布偏度，小于零时左偏，大于零时右偏；

In [57]:
a = np.array([[1, 2, 3, 2, 5, 1], [0, 0, 0, 1, 1, 1]])
df = pd.DataFrame(a.T, columns=['x1', 'x2'])
df

,x1,x2
0,1,0
1,2,0
2,3,0
3,2,1
4,5,1
5,1,1


In [58]:
aggs = {'x1': ['mean', 'var', 'max', 'min', 'skew']}
df.groupby('x2').agg(aggs).reset_index()

x2        x1                            
          mean       var max min      skew
0  0  2.000000  1.000000   3   1  0.000000
1  1  2.666667  4.333333   5   1  1.293343

常用的分类变量的统计量如下，当然除了偏度外，其他连续变量的统计量也是可用于分类变量的：

    - median：中位数；
    - count：个数统计；
    - nunique：类别数；
    - quantile：分位数

In [254]:
df = pd.DataFrame({'x1':[1, 3, 4, 2, 1], 'x2':[0, 0, 1, 1, 1]})
df

,x1,x2
0,1,0
1,3,0
2,4,1
3,2,1
4,1,1


In [255]:
aggs = {'x1': ['median', 'count', 'nunique']}
df.groupby('x2').agg(aggs).reset_index()

x2     x1              
     median count nunique
0  0      2     2       2
1  1      2     3       3

而对于分位数的计算，则需要借助自定义函数来完成计算：

In [23]:
def q1(x):
    """
    下四分位数
    """
    return x.quantile(0.25)

def q2(x):
    """
    上四分位数
    """
    return x.quantile(0.75)

In [33]:
d1 = pd.DataFrame({'x1':[3, 2, 4, 4, 2, 2], 'x2':[0, 1, 1, 0, 0, 0]})
d1

,x1,x2
0,3,0
1,2,1
2,4,1
3,4,0
4,2,0
5,2,0


In [34]:
aggs = {'x1': [q1, q2]}
d2 = d1.groupby('x2').agg(aggs).reset_index()
d2

x2   x1      
       q1    q2
0  0  2.0  3.25
1  1  2.5  3.50

In [35]:
d2.columns = ['x2', 'x1_x2_q1', 'x1_x2_q2']
d2

,x2,x1_x2_q1,x1_x2_q2
0,0,2.0,3.25
1,1,2.5,3.50


当然，分位数也是可以应用于连续变量的。

&emsp;&emsp;最后，让我们来汇总各种不同类型的变量可以使用的统计量。正如此前讨论的，分位数可以用于连续变量，而连续变量的统计指标中只有偏度不适用于离散变量。据此我们在划分连续变量和分类变量后，可以设置如下基本统计衍生指标：

In [275]:
aggs_num = {'num': ['mean', 'var', 'max', 'min', 'skew', 'median', 'q1', 'q2']}

In [276]:
aggs_cat = {'cat': ['mean', 'var', 'max', 'min', 'median', 'count', 'nunique', 'q1', 'q2']}

> 需要注意的是，上面的统计指标设置只适用于一般情况，在某些情况下，如连续变量取值个数较少（只有十几个或者几十个不同取值）时，该连续变量也可以使用'count'、'nunique'等指标，而如果分类变量取值个数较多（如超过5个），则也可以使用偏度计算公式，具体如何选择还需要视具体情况而定。

### 2.多变量的分组统计特征衍生

- 方法介绍

&emsp;&emsp;接下来，进一步考虑多变量分组特征衍生的方法。在双变量分组特征衍生时，我们是选择某个特征为KeyCol（关键特征），然后以KeyCol的不同取值为作为分组依据，计算其他特征的统计量。而在多变量分组特征衍生的过程中，我们将考虑采用不同离散变量的交叉组合后的取值分组依据，再进行分组统计量的计算。在双变量分组统计汇总下，基本计算过程如下：

<center><img src="https://s2.loli.net/2022/01/20/6YzMGo9uI1Dm8rc.png" alt="image-20220120170224947" style="zoom:33%;" />

而多变量分组统计汇总基本过程如下：

<center><img src="https://s2.loli.net/2022/02/11/7wgWCorpUuLEKTS.png" alt="image-20220211190335669" style="zoom:33%;" />

此处是以tenure和SeniorCitizen交叉组合后的结果作为分组依据，对Monthly Charges进行分组汇总。除了分组的依据发生了变化外，分组统计过程和此前介绍的双变量分组统计特征衍生过程并没有任何差异。当然，从直观的结果上来看，多变量分组统计特征衍生能够更细粒度的呈现数据集信息，例如在以tenure作为分组依据统计Monthly Charges时相当于计算不同入网时间用户的平均月消费金额（以mean为例），而如果是以tenure与SeniorCitizen交叉组合结果作为分组依据统计Monthly Charges时，则相当于是计算不同入网时间、不同年龄段用户的平均消费金额，而往往更细粒度的信息展示就能够帮助模型达到更好的效果，因此，有限范围内的多变量分组统计特征衍生，是能达到更好的效果的。但同时需要注意的是，这种“细粒度”的呈现并不是越细粒度越好，我们知道，参与分组的交叉特征越多、分组也就越多，而在相同数据集下，分组越多、每一组的组内样本数量就越少，而在进行组内统计量计算时，如果组内样本数量太少，统计量往往就不具备代表性了，例如上述极简示例中ID为1的样本，在tenure和SeniorCitizen交叉分组后该样本所属分组只有一条样本，后续计算的统计量也没有任何“统计”方面的价值了。因此，多变量交叉分组也并不是越多变量越细粒度越好。一般来说，对于人工判断极重要的特征，可以考虑两个或三个特征进行交叉组合后分组。

### 5.统计演变特征

- 二阶特征衍生

&emsp;&emsp;一种很自然的联想，是当我们已经完成了一些特征衍生后，还会考虑以衍生特征为基础，进一步进行特征衍生，这也就是所谓的二阶特征衍生（注意区分二阶多项式衍生）。当然这个过程可以无限重复，这也是此前讨论为何会出现无限特征的根本原因之一。不过，在大多数情况下，二阶甚至是更高阶的特征衍生（以下简称高阶特征衍生）往往伴随着严重的信息衰减，大多数高阶衍生出来的特征其本身的有效性也将急剧下降，外加高阶特征衍生是在已有的大量衍生出来的一阶特征基础上再进行衍生，其计算过程往往需要消耗巨大的计算量，外加需要从一系列高阶衍生特征中挑选出极个别有用的特征也较为繁琐，因此，高阶衍生往往性价比较低，除非特殊情况，否则并不建议在广泛特征基础上进行大量高阶特征衍生的尝试。

- 统计演变特征

&emsp;&emsp;当然，尽管并不建议手动进行尝试，但在长期的实践过程中，人们还是总结出某些高阶衍生特征（主要是二阶衍生特征）在很多情况下都能起到很好的效果。需要注意的是，这里的着重指的是一些特征，而不是特征衍生的策略。在这些普遍有效的高阶特征中，最著名的就是所谓的统计演变特征，这些特征由原始特征和分组统计特征、或分组统计特征彼此之间交叉衍生而来，在很多算法竞赛和企业应用中，都被证明了有较高的尝试价值。

&emsp;&emsp;接下来我们就对这些统计演变特征进行逐一介绍。

#### 5.1 原始特征与分组汇总特征交叉衍生

&emsp;&emsp;在统计演变特征中，最常用的特征衍生方法就是利用KeyCol和分组统计衍生特征进行交叉衍生，例如此前数据集分组统计汇总衍生为例，此处我们以tenure作为分组依据，对月均消费金额进行分组汇总统计，有计算结果如下：

In [124]:
col_num = ['MonthlyCharges']
df, col = Binary_Group_Statistics(keyCol, features, col_num)
df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,MonthlyCharges_tenure_mean,MonthlyCharges_tenure_var,MonthlyCharges_tenure_max,MonthlyCharges_tenure_min,MonthlyCharges_tenure_skew,MonthlyCharges_tenure_median
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,Yes,Electronic check,29.85,29.85,50.485808,610.791587,102.45,18.80,0.092226,49.750
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,Mailed check,56.95,1889.50,69.644615,866.891416,116.25,19.60,-0.384323,73.950
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,Yes,Mailed check,53.85,108.15,57.206303,634.068336,104.40,18.75,-0.220111,61.075
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,Bank transfer (automatic),42.30,1840.75,71.245902,938.289858,115.65,18.85,-0.481543,81.000
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,Yes,Electronic check,70.70,151.65,57.206303,634.068336,104.40,18.75,-0.220111,61.075


我们依此为依据，可以进一步构建下列统计演变特征：

- 流量平滑特征

&emsp;&emsp;该特征通过KeyCol除以分组汇总均值后的特征计算而来，也就是利用tenure除以MonthlyCharges_tenure_mean计算得出。当然因为是进行除法运算，为了避免分母为零的情况，我们可以在分母位上加上一个很小的数，具体计算过程如下：

In [129]:
df['tenure'] / (df['MonthlyCharges_tenure_mean'] + 1e-5)

0       0.019808
1       0.488193
2       0.034961
3       0.631615
4       0.034961
          ...   
7038    0.391239
7039    0.892239
7040    0.188122
7041    0.069647
7042    0.867696
Length: 7043, dtype: float64

- 黄金组合特征

&emsp;&emsp;所谓黄金组合特征，就是简单的利用tenure减去MonthlyCharges_tenure_mean计算得出：

In [130]:
df['tenure'] - df['MonthlyCharges_tenure_mean']

0      -49.485808
1      -35.644615
2      -55.206303
3      -26.245902
4      -55.206303
          ...    
7038   -37.343617
7039    -8.695856
7040   -47.472727
7041   -53.432670
7042   -10.063483
Length: 7043, dtype: float64

- 组内归一化特征

&emsp;&emsp;所谓组内归一化特征，指的是用tenure减去MonthlyCharges_tenure_mean，再除以MonthlyCharges_tenure_std，其计算过程非常类似于归一化过程，即某列数据减去该列的均值再除以该列的标准差，这也是组内归一化名称的由来。具体计算过程如下：

In [135]:
(df['tenure'] - df['MonthlyCharges_tenure_mean']) / (np.sqrt(df['MonthlyCharges_tenure_var']) + 1e-5)

0      -2.002322
1      -1.210630
2      -2.192403
3      -0.856826
4      -2.192403
          ...   
7038   -1.317354
7039   -0.272113
7040   -1.741320
7041   -2.026832
7042   -0.339537
Length: 7043, dtype: float64

&emsp;&emsp;不难看出这些衍生过程仍然还是主要用到四则运算衍生方法，其计算过程并不复杂，而在实际操作过程中需要注意的是，往往需要同时带入基础的分组汇总衍生的特征和上述二阶衍生特征，才能起到更好的效果。

#### 5.2 分组汇总特征彼此交叉衍生

&emsp;&emsp;另外一类常用的二阶衍生特征，就是一系列基于分组汇总统计后的信息再次进行交叉衍生得到的新特征。这类特征往往具有较强的统计背景，能够更好的衡量原始特征的基本分布情况，还是在上述数据集中，以tenure作为分组依据，对月均消费金额进行分组汇总统计后，我们可以围绕这些统计指标进行二阶衍生：

In [43]:
col_num = ['MonthlyCharges']
df, col = Binary_Group_Statistics(keyCol, features, col_num)
df.head(5)

,MonthlyCharges_tenure_mean,MonthlyCharges_tenure_var,MonthlyCharges_tenure_max,MonthlyCharges_tenure_min,MonthlyCharges_tenure_skew,MonthlyCharges_tenure_median,MonthlyCharges_tenure_q1,MonthlyCharges_tenure_q2
0,50.485808,610.791587,102.45,18.80,0.092226,49.750,20.9,71.35
1,69.644615,866.891416,116.25,19.60,-0.384323,73.950,50.2,94.25
2,57.206303,634.068336,104.40,18.75,-0.220111,61.075,34.8,79.15
3,71.245902,938.289858,115.65,18.85,-0.481543,81.000,50.9,96.75
4,57.206303,634.068336,104.40,18.75,-0.220111,61.075,34.8,79.15


- Gap特征

&emsp;&emsp;Gap特征通过分组汇总后的上四分位数-下四分位数计算得出。在此前使用groupby进行分组统计的过程中我们并未使用分位数作为统计指标，具体分组分位数的计算过程需要借助，然后再带入groupby的过程：

当然，相同的计算过程也可应用于此前数据集中MonthlyCharges在不同tenure取值下的计算：

In [160]:
aggs = {'MonthlyCharges': [q1, q2]}
features_temp = features.groupby('tenure').agg(aggs).reset_index()
features_temp.head(5)

tenure MonthlyCharges          
                      q1        q2
0       0        20.1250   58.9750
1       1        20.9000   71.3500
2       2        34.8000   79.1500
3       3        29.8750   79.2875
4       4        29.5250   79.3375
..    ...            ...       ...
68     68        54.3250   97.3000
69     69        32.7750  101.7250
70     70        58.3750  103.4750
71     71        47.1875   99.6500
72     72        64.8375  108.2750

[73 rows x 3 columns]

In [161]:
features_temp.columns = ['tenure', 'MonthlyCharges_tenure_q1', 'MonthlyCharges_tenure_q2']
features_temp.head(5)

,tenure,MonthlyCharges_tenure_q1,MonthlyCharges_tenure_q2
0,0,20.125,58.9750
1,1,20.900,71.3500
2,2,34.800,79.1500
3,3,29.875,79.2875
4,4,29.525,79.3375


In [163]:
features_temp['MonthlyCharges_tenure_q2-q1'] = features_temp['MonthlyCharges_tenure_q2'] - features_temp['MonthlyCharges_tenure_q1']
features_temp.head(5)

,tenure,MonthlyCharges_tenure_q1,MonthlyCharges_tenure_q2,MonthlyCharges_tenure_q2-q1
0,0,20.125,58.9750,38.8500
1,1,20.900,71.3500,50.4500
2,2,34.800,79.1500,44.3500
3,3,29.875,79.2875,49.4125
4,4,29.525,79.3375,49.8125


- 数据倾斜

&emsp;&emsp;此外，我们还可以通过中位数和均值的比较来计算组内的数据倾斜情况：当均值大于中位数时，数据呈现正倾斜，均值小于中位数时，数据正弦负倾斜。当然衡量倾斜的方法有两种，其一是计算差值，其二则是计算比值：

In [164]:
col_num = ['MonthlyCharges']
df, col = Binary_Group_Statistics(keyCol, features, col_num)
df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,MonthlyCharges_tenure_mean,MonthlyCharges_tenure_var,MonthlyCharges_tenure_max,MonthlyCharges_tenure_min,MonthlyCharges_tenure_skew,MonthlyCharges_tenure_median
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,Yes,Electronic check,29.85,29.85,50.485808,610.791587,102.45,18.80,0.092226,49.750
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,Mailed check,56.95,1889.50,69.644615,866.891416,116.25,19.60,-0.384323,73.950
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,Yes,Mailed check,53.85,108.15,57.206303,634.068336,104.40,18.75,-0.220111,61.075
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,Bank transfer (automatic),42.30,1840.75,71.245902,938.289858,115.65,18.85,-0.481543,81.000
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,Yes,Electronic check,70.70,151.65,57.206303,634.068336,104.40,18.75,-0.220111,61.075


In [165]:
df['MonthlyCharges_tenure_mean'] - df['MonthlyCharges_tenure_median']

0       0.735808
1      -4.305385
2      -3.868697
3      -9.754098
4      -3.868697
          ...   
7038    1.943617
7039   -8.379144
7040   -2.777273
7041    0.457670
7042   -4.486517
Length: 7043, dtype: float64

In [167]:
df['MonthlyCharges_tenure_mean'] / (df['MonthlyCharges_tenure_median'] + 1e-5)

0       1.014790
1       0.941780
2       0.936656
3       0.879579
4       0.936656
          ...   
7038    1.032721
7039    0.905931
7040    0.954657
7041    1.008033
7042    0.944301
Length: 7043, dtype: float64

- 变异系数

&emsp;&emsp;变异系数是通过分组统计的标准差除以均值，变异系数计算的是离中趋势，变异系数越大、说明数据离散程度越高，相关计算过程如下：

In [168]:
np.sqrt(df['MonthlyCharges_tenure_var']) / (df['MonthlyCharges_tenure_mean'] + 1e-10)

0       0.489528
1       0.422761
2       0.440174
3       0.429941
4       0.440174
          ...   
7038    0.462109
7039    0.396015
7040    0.466243
7041    0.459018
7042    0.389659
Length: 7043, dtype: float64